In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
path_data = os.getenv("path_to_data")
path_master = os.getenv("path_to_master")

## Wetter Komplett

In [3]:
# Wetterdaten und E. coli-Messungen laden
df_wetter = pd.read_csv(f"{path_data}/clean_wetter_komplett.csv", header=0, index_col=0, parse_dates=True)
df_messungen = pd.read_csv(f"{path_master}/messungen_clean_2024.csv", header=0, index_col=0, parse_dates=True)


In [4]:
df_komplett = pd.merge(df_wetter, df_messungen, how="left", left_index=True, right_index=True)
df_komplett.index.rename("zeit", inplace=True)

In [5]:
df_komplett = df_komplett[df_komplett.index >= "2024-01-01"]

In [6]:
# LUBW-Daten laden und verarbeiten
df_lubw_raw = pd.read_csv(f"{path_data}/messwerte_lubw.csv", sep=";", parse_dates=["Datum"], dayfirst=True)

# Relevante Spalten extrahieren
df_lubw = df_lubw_raw[["Messstation", "Gewässer", "Parameter", "Datum", "Tagesmittelwert"]]

In [7]:
# Zahlen konvertieren (Komma → Punkt, "-" → NaN)
df_lubw["Tagesmittelwert"] = df_lubw["Tagesmittelwert"].str.replace(",", ".")
df_lubw["Tagesmittelwert"] = pd.to_numeric(df_lubw["Tagesmittelwert"], errors="coerce")


C:\Users\Dome Arbeit\AppData\Local\Temp\ipykernel_20904\1829527460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lubw["Tagesmittelwert"] = df_lubw["Tagesmittelwert"].str.replace(",", ".")
C:\Users\Dome Arbeit\AppData\Local\Temp\ipykernel_20904\1829527460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lubw["Tagesmittelwert"] = pd.to_numeric(df_lubw["Tagesmittelwert"], errors="coerce")


In [8]:
# Kürzel erstellen z.B. We_Ne_Temperatur
df_lubw["Kürzel"] = (
    df_lubw["Messstation"].str[:2].str.capitalize() + "_" +
    df_lubw["Gewässer"].str[:2].str.capitalize() + "_" +
    df_lubw["Parameter"]
        .str.replace("bei .*", "", regex=True)
        .str.replace(" ", "")
        .str.replace("ä", "ae")
        .str.replace("ö", "oe")
        .str.replace("ü", "ue")
        .str.replace("ß", "ss")
)


C:\Users\Dome Arbeit\AppData\Local\Temp\ipykernel_20904\133331347.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lubw["Kürzel"] = (


In [9]:
# Pivotieren: Datum als Index, Parameter als Spalten
df_lubw_pivot = df_lubw.pivot(index="Datum", columns="Kürzel", values="Tagesmittelwert")

# Resample auf 1h und auffüllen
df_lubw_hourly = df_lubw_pivot.resample("1H").ffill()

# In Masterdaten integrieren
df_komplett = df_komplett.merge(df_lubw_hourly, left_index=True, right_index=True)

C:\Users\Dome Arbeit\AppData\Local\Temp\ipykernel_20904\4046771336.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_lubw_hourly = df_lubw_pivot.resample("1H").ffill()


In [10]:
# Formatierung
df_komplett.reset_index(inplace=True)
df_komplett.columns = [col.strip() for col in df_komplett.columns]
df_komplett.set_index("index", inplace=True)

In [11]:
# Export
df_komplett.to_csv(f"{path_master}/masterdata_2024.csv")